The following logic are strictly only for H3 candles, three-hour interval bars. {

Original entry: {If the '0:00 am to 3:00 am H3 bar' is bullish, then market-buy exactly at its close (3:00 am). The 'initial stop-loss' it at the low of '0:00 am to 3:00 am H3 bar'. Risking 1% on the trade accounting the full stop-loss distance. The take-profit will depend on the stop-loss trailing mechanic. The stoploss will only trail at every proceeding 'bullish H3 candle's low', skip the bearish H3 candles. The trailing mechanic will only apply after the H3 close. The stoploss trailing is not limited to number of candles; it can run infinitely.}

Reverse entry: {If the 'Original entry' closed (stopped out) at negative stoploss (initial-stoploss or trailed-stoploss as long as it's not in profit), then 'market-sell exactly at the stop out price'. Locate the position -1 candle's high and place the 'initial-stoploss' there. And also, risking 1% on the trade accounting the full stop-loss distance. And also, the take-profit will depend on the stop-loss trailing mechanic. And also, the stoploss will only trail at every proceeding 'bearish H3 candle's high', skip the bullish H3 candles. The trailing mechanic will only apply after the H3 close. The stoploss trailing is not limited to number of candles; it can run infinitely.}

And on the other side,

Original entry: {If the '0:00 am to 3:00 am H3 bar' is bearish, then market-sell exactly at its close (3:00 am). The 'initial stop-loss' it at the high of '0:00 am to 3:00 am H3 bar'. Risking 1% on the trade accounting the full stop-loss distance. The take-profit will depend on the stop-loss trailing mechanic. The stoploss will only trail at every proceeding 'bearish H3 candle's high', skip the bullish H3 candles. The trailing mechanic will only apply after the H3 close. The stoploss trailing is not limited to number of candles; it can run infinitely.}

Reverse entry: {If the 'Original entry' closed (stopped out) at negative stoploss (initial-stoploss or trailed-stoploss as long as it's not in profit), then 'market-buy exactly at the stop out price'. Locate the position -1 candle's low and place the 'initial-stoploss' there. And also, risking 1% on the trade accounting the full stop-loss distance. And also, the take-profit will depend on the stop-loss trailing mechanic. And also, the stoploss will only trail at every proceeding 'bullish H3 candle's low', skip the bearish H3 candles. The trailing mechanic will only apply after the H3 close. The stoploss trailing is not limited to number of candles; it can run infinitely.}

}

